In [2]:
pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 623.7 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import ast
import os
from graphviz import Digraph

def parse_functions(file_content, file_name):
    tree = ast.parse(file_content)
    functions = {}
    
    class FunctionVisitor(ast.NodeVisitor):
        def visit_FunctionDef(self, node):
            functions[node.name] = {
                'file': file_name,
                'calls': [n.func.id for n in ast.walk(node) if isinstance(n, ast.Call) and isinstance(n.func, ast.Name)]
            }
            self.generic_visit(node)
    
    FunctionVisitor().visit(tree)
    return functions

def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def parse_directory(directory):
    all_functions = {}
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                file_content = read_file(file_path)
                file_functions = parse_functions(file_content, file)
                all_functions.update(file_functions)
    return all_functions

def generate_explanation(functions):
    explanations = []
    for func, details in functions.items():
        if details['calls']:
            explanations.append(f"The function '{func}' in file '{details['file']}' calls the functions: {', '.join(details['calls'])}.")
        else:
            explanations.append(f"The function '{func}' in file '{details['file']}' does not call any other functions.")
    return explanations

def create_diagram(functions, output_file='diagram'):
    dot = Digraph(comment='Function Call Graph')
    for func, details in functions.items():
        dot.node(func, f"{func} ({details['file']})")
        for called_func in details['calls']:
            if called_func in functions:
                dot.edge(func, called_func)
    dot.render(output_file, format='png')
    print(f"Diagram saved as {output_file}.png")

# Example usage for the entire directory
directory_path = 'employee-demo-app'
all_functions = parse_directory(directory_path)

explanations = generate_explanation(all_functions)
for explanation in explanations:
    print(explanation)

create_diagram(all_functions)


The function 'get_companies' in file 'company_service.py' calls the functions: get_company_info, jsonify.
The function 'get_company' in file 'company_service.py' calls the functions: get_company_info, jsonify, jsonify.
The function 'get_company_info' in file 'utils.py' does not call any other functions.
The function 'get_employees' in file 'employee_service.py' calls the functions: get_employee_info, jsonify.
The function 'get_employee' in file 'employee_service.py' calls the functions: get_employee_info, jsonify, jsonify.
The function 'get_employee_info' in file 'utils.py' does not call any other functions.
Diagram saved as diagram.png
